In [1]:
import re
from functools import cache, cached_property
from collections import deque
from math import ceil

with open('./assets/input_day_19.txt','r') as file:
    input = {}
    temp = [tuple([int(x) for x in re.findall(r"(\d+)",line)]) for line in file.read().splitlines()]
    for line in temp:
        input[line[0]] = line[1:]

        
class Blueprint(object):
    def __init__(self, blueprint, total_time = 24, robots = (1,0,0,0), resources = (0,0,0,0)) -> None:
        self._bp = blueprint
        self.robots = robots
        self.resources = resources
        self._total_time = total_time


    @cached_property
    def blueprint(self):
        return [
            (self._bp[0],0,0,0),
            (self._bp[1],0,0,0),
            (self._bp[2],self._bp[3],0,0),
            (self._bp[4],0,self._bp[5],0)
        ]

    @cached_property
    def max_bots(self):
        return [max(cost[i] for cost in self.blueprint) for i in range(4)]

    @cache
    def can_build(self, type, resources):
        return all([x - y >= 0 for x,y in zip(resources,self.blueprint[type])])

    def bots_to_build(self,type,robots):
        return all([y != 0 for x,y in zip(self.blueprint[type],robots) if x])


    @cache
    def build_options(self,resources,robots, time):

        output = []

        for bid in range(4):
            if bid != 3 and robots[bid] >= self.max_bots[bid]:
                continue

            if any(robots[rid] == 0 for rid, cost in enumerate(self.blueprint[bid]) if cost):
                continue 

            wait = max([ceil((cost - resources[rid]) / robots[rid])  for rid,cost in enumerate(self.blueprint[bid]) if cost] + [0])
            if time - wait - 1 <= 0:
                continue

            output.append((bid,wait))

        return output


    def solve(self):


        queue = deque()
        queue.append((self._total_time,self.resources,self.robots))
        seen = set()
        best = 0

        while queue:
            t, resources, robots = queue.popleft()

            if (m := resources[3] + t * robots[3]) > best:
                best = m
            
            if t == 0 or (s := (t, resources, robots)) in seen:
                continue
            else:
                seen.add(s)

            for bid, wait in self.build_options(resources,robots,t):

                new_resources =  [resources[i] + (robots[i] * (wait + 1)) - self.blueprint[bid][i] for i in range(4)]

                for i in range(3):
                    new_resources[i] = min(new_resources[i], self.max_bots[i] * (t - wait - 1))
                    
                new_robots = list(robots)
                new_robots[bid] += 1

                queue.append((t - wait -1, tuple(new_resources),tuple(new_robots)))

        return best


In [2]:
quality = 0
for k,v in input.items():
    a = Blueprint(v)
    quality += a.solve()*k

print(quality)


1294


In [4]:
quality = 1
for i in range(3):
    a = Blueprint(input[i+1],total_time=32)
    quality *= a.solve()

In [5]:
print(quality)

13640
